In [73]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 300)
pd.set_option('max_columns', 200)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

startDate = '20201204'
endDate = '20201207'

readPath = r'\\192.168.10.28\equityTradeLogs'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]

rawOrderLog = []
for thisPath in dataPathLs:
    data = pd.read_csv(thisPath)
    data = data.rename(columns={'mdClockAtArrival': 'caamd'})
    rawOrderLog += [data]
rawOrderLog = pd.concat(rawOrderLog, sort=False)

for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].fillna(0).astype('int64')   
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

if rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)].shape[0] != 0:
    display('There are accounts with duplicated ticks:')
    display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)]\
.groupby(['date', 'colo', 'accCode'])['ars'].size())
    rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep='first')

display('There are ticks with orderDirection 0')
display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
try:
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
except:
    print('There are orders with all things same except sdd')
    print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)])
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
try:
    assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
except:
    print('There are orders with same internalId but different orderId other than accCode 8856 case')
    print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize']).grouper.group_info[0]
r1 = r1.sort_values(by=['test', 'clockAtArrival'])
r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
r1['vai'] = r1.groupby('test')['vai'].ffill()
r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
r2 = r2.sort_values(by=['test', 'clockAtArrival'])
r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
r2['vai'] = r2.groupby('test')['vai'].ffill()
assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
try:
    assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
orderLog = pd.concat([r1, r2])
del r1
del r2    

orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
orderLog['broker'] = orderLog['accCode'] // 100
orderLog["broker"] = np.where(orderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), orderLog['accCode'] // 10000, orderLog["broker"])
orderLog['colo_broker'] = orderLog['colo'].str[:2] + '_' + orderLog['broker'].astype('str')
orderLog['order'] = orderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai']).grouper.group_info[0]
orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
    orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')


### Assertion 1:  make sure same direction in same date, secid, vai
print('=======================================================================================')
print('1. same date, secid, vai: same direction')
orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
if len(orderLog[orderLog['directNum'] != 1]) > 0:
    print('opposite direction for same date, same secid, same vai')
    display(orderLog[(orderLog['directNum'] != 1) & (orderLog['updateType'] == 0)].groupby(['accCode'])['orderDirection'].size())
    orderLog = orderLog[orderLog['directNum'] == 1]

assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].nunique() == 1).all() == True)

## Assertion 2:  make sure each account, secid, vai only has one insertion
print('=======================================================================================')
print('2. same date, secid, vai, accCode: one insertion')
a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
if len(a[a > 1]) > 0:
    print('more than one insertion at same time')
    a = a[a>1].reset_index()
    display(a)
    display(a.groupby(['accCode'])['date'].size())
    orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                             np.where(orderLog['mse'] == 100, 1, 0), np.nan)
orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))


### Assertion 3:  check IPO stocks selling status
print('=======================================================================================')
print('3. IPO stocks selling (ars = 301, 302)')
if orderLog[orderLog['ars'].isin([301, 302])].shape[0] != 0:
    kk = orderLog[orderLog['ars'].isin([301, 302])]
    print(kk)
    try:
        assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
        print('we only sell, never buy')
    except:
        print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk[kk['orderDirection1'] == 1])
    kk1 = kk[kk['updateType'] == 0]
    kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
    kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
    kk1['diff'] = kk1['diff'].fillna(0)
    try:
        assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
        print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
    except:
        print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk1[kk1['diff'] < 10e6])
    kk2 = kk[(kk['updateType'] == 1)]
    try:
        assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
        print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
    except:
        print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk2[kk2['duration'] < 3e6])


### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
print('=======================================================================================')
print('4. updateType 7 orders')
if orderLog[orderLog['updateType'] == 7].shape[0] != 0:
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().max() < 20)
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().sum() < 100)

### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
print('=======================================================================================')
print('5. updateType 6 orders')
k1 = orderLog[orderLog['updateType'] == 6].groupby(['date', 'accCode'])['order'].nunique().reset_index()
k2 = orderLog.groupby(['date', 'accCode'])['order'].nunique().reset_index()
k = pd.merge(k1, k2, on=['date', 'accCode'], how='left')
k['prob'] = k['order_x']/k['order_y']
try:
    assert(sum(k['prob'] >= 0.05) == 0)
except:
    print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(k[k['prob'] >= 0.05])

### Assertion 6: check CYB orders, make sure all CYB stocks have absOrderSize < 30w
print('=======================================================================================')
print('6. CYB stocks order size < 30w')
try:
    cyb = orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0)].groupby('date')['absOrderSize'].max().reset_index()
    assert(cyb[cyb['absOrderSize'] > 300000].shape[0] == 0)
except:
    print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    
     
### Assertion 7:  make sure there is no unexpected updateType 
print('=======================================================================================')
print('7. unexpected updateType')
def getTuple(x):
    return tuple(i for i in x)

checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
                                                               (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
                                                               (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
                     np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))

orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

### Assertion 8:  make sure status==0 got all traded
print('=======================================================================================')
print('8. status == 0: all traded')
a = orderLog[orderLog['status'] == 0]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in total trade, any fill != total cases')
display(a[a['filled'] != a['total']])
if a[a['filled'] != a['total']].shape[0] > 0:
    removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 9:  make sure status==1 got partial traded
print('=======================================================================================')
print('9. status == 1: partial traded')
a = orderLog[orderLog['status'] == 1]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
    removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 10: make sure no cancellation within 1 sec
print('=======================================================================================')
print('10. no cancellation within 1 sec')
a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
print('any cancellation within 1 sec')
display(a)
if a.shape[0] > 0:
    removeOrderLs = a['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


### Assertion 11: make sure no order has shares > 80w or notional > 800w
print('=======================================================================================')
print('11. Orders with size > 80w or notional > 800w')
orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
    print('some order quantity are > 80w')
    print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                         'orderNtl', 'orderDirection', 'clock', 'order']])
            
if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
    print('some order ntl are > 800w')
    print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                      'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                      "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

orderLog["mrsb90"] = orderLog.groupby(['order'])['mrsb90'].transform('first')
orderLog["mrss90"] = orderLog.groupby(['order'])['mrss90'].transform('first')
orderLog["aaa"] = orderLog.groupby(['order'])['aaa'].transform('first')

orderLog['m1'] = orderLog['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
orderLog['m2'] = orderLog['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
if orderLog[orderLog['mrsb90'] == '-'].shape[0] != 0:
    display(orderLog[orderLog['mrsb90'] == '-'])
orderLog = orderLog[orderLog['mrsb90'] != '-']
orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
if orderLog[orderLog['aaa'] == '-'].shape[0] != 0:
    display(orderLog[orderLog['aaa'] == '-'])
orderLog = orderLog[orderLog['aaa'] != '-']
orderLog['aaa'] = orderLog['aaa'].astype(float)
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm1']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm1']    

orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
orderLog["mrstaat"] = orderLog.groupby(['order'])['mrstaat'].transform('first')
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
orderLog["mrstauc"] = orderLog.groupby(['order'])['mrstauc'].transform('first')
orderLog = orderLog[~orderLog['ars'].isnull()]
orderLog['sta'] = np.where(orderLog['mrstaat'] == 1000, '1. staone', np.where(
orderLog['mrstaat'] == 3000, '2. statwo', np.where(
orderLog['mrstaat'].isin([11000, 13000]), '3. sta300', 'else')))
display(orderLog[orderLog['sta'] == 'else'][['date', 'secid', 'colo', 'accCode', 'updateType', 'vai', 'absOrderSize', 'orderDirection', 'mrstaat', 'mrstauc', 'sdd', 'ApplSeqNum', 'sta']])
orderLog = orderLog[orderLog['mrstaat'].isin([11000, 13000, 1000, 3000])]
display(orderLog[orderLog['updateType'] == 0].groupby(['mrstaat', 'mrstauc'])['date'].size())

'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
230972,20201204,zs_96_08,9685,2000403,-1,1,50390.0,0,0,-1.0,7.221449e+17
236722,20201204,zs_64_01,6480,2000545,-1,7,35341.0,0,0,-1.0,-1.000000e+00
246097,20201204,zs_52_06,5269,2000659,-1,1,52696.0,0,0,-1.0,1.079832e+06
259788,20201204,zs_54_01,5474,2000828,-1,1,53258.0,0,0,-1.0,8.689151e+08
295035,20201204,zs_52_06,5269,2002173,-1,1,47904.0,0,0,-1.0,8.682430e+05
312827,20201204,zs_54_01,5474,2002297,-1,1,48345.0,0,0,-1.0,8.689114e+08
343322,20201204,zs_54_01,5474,2002553,-1,1,36111.0,0,0,-1.0,8.689046e+08
358055,20201204,zs_52_06,5269,2002645,-1,1,40833.0,0,0,-1.0,7.924330e+05
401609,20201204,zs_52_06,5269,2002865,-1,1,39775.0,0,0,-1.0,7.313410e+05
409854,20201204,zs_66_01,6634,2002917,-1,1,41354.0,0,0,-1.0,1.812806e+10


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


3. IPO stocks selling (ars = 301, 302)
4. updateType 7 orders
5. updateType 6 orders
6. CYB stocks order size < 30w
7. unexpected updateType
8. status == 0: all traded
in total trade, any fill != total cases


,order,filled,total
4083,5881,100,500
16516,22389,9500,15000
18248,24923,50,600
19611,26742,200,400
25197,34442,1503,3300
27162,36745,400,900
34690,46349,1600,2300
42383,55413,700,900
67881,87287,100,200
82207,106387,2100,5300


...

10. no cancellation within 1 sec
any cancellation within 1 sec


,ApplSeqNum,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,test,broker,colo_broker,order,group,startClock,duration,orderDirection1,status


11. Orders with size > 80w or notional > 800w


,date,secid,colo,accCode,updateType,vai,absOrderSize,orderDirection,mrstaat,mrstauc,sdd,ApplSeqNum,sta
280794,20201204,2002075,zs_88_04,8856,0,36818706.0,400,-1,NaN,NaN,145617040.0,NaN,else
675678,20201207,1601686,zt_88_06,8854,0,302.0,735,-1,0.0,0.0,134146000.0,0.0,else
675679,20201207,1601686,zt_88_06,8854,2,302.0,735,-1,0.0,0.0,-1.0,0.0,else
675680,20201207,1601686,zt_88_06,8854,4,302.0,735,-1,0.0,0.0,-1.0,0.0,else
675681,20201207,1601686,zt_88_06,8886,0,302.0,735,-1,0.0,0.0,134146000.0,0.0,else
675682,20201207,1601686,zt_88_06,8886,2,302.0,735,-1,0.0,0.0,-1.0,0.0,else
675683,20201207,1601686,zt_88_06,8886,4,302.0,735,-1,0.0,0.0,-1.0,0.0,else
675684,20201207,1601686,zs_94_03,9461,0,302.0,707,-1,0.0,0.0,134146000.0,0.0,else
675685,20201207,1601686,zs_94_03,9461,2,302.0,707,-1,0.0,0.0,-1.0,0.0,else
675686,20201207,1601686,zs_94_03,9461,4,302.0,707,-1,0.0,0.0,-1.0,0.0,else


mrstaat  mrstauc
1000.0   0.0         44024
3000.0   1000.0     180529
         2000.0      11882
         3000.0       4546
11000.0  10000.0       712
         20000.0        64
         30000.0        54
13000.0  11000.0     16571
         12000.0       751
         13000.0       221
         21000.0       473
         22000.0       532
         23000.0       107
         31000.0       232
         32000.0       189
         33000.0       259
Name: date, dtype: int64

In [6]:
orderLog[(orderLog['updateType'] == 0) & (orderLog['accCode'] == 5456) & (orderLog['ars'] == 7)].shape[0] / \
orderLog[(orderLog['updateType'] == 0) & (orderLog['accCode'] == 5456)].shape[0]

0.29782833505687695

In [13]:
1 - orderLog[(orderLog['ars'] == 7) & (orderLog['updateType'] == 1)].shape[0] / orderLog[(orderLog['ars'] == 7) & (orderLog['updateType'] == 0)].shape[0]

0.9930555555555556

In [14]:
1 - orderLog[(orderLog['ars'] == 1) & (orderLog['updateType'] == 1) & (orderLog['accCode'] == 5456)].shape[0] / orderLog[(orderLog['ars'] == 1) & (orderLog['updateType'] == 0) & (orderLog['accCode'] == 5456)].shape[0]

0.8792341678939617

In [15]:
1 - orderLog[(orderLog['updateType'] == 1) & (orderLog['accCode'] == 5456)].shape[0] / orderLog[(orderLog['updateType'] == 0) & (orderLog['accCode'] == 5456)].shape[0]

0.9131334022750776

In [54]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import pickle
from IPython.display import display, HTML
# 上次遗留问题：调整图间间距
# 调整标签位置，不要和图中重合
startDate = '20201204'
endDate = '20201207'

readPath = "L:\\orderLog\\ret\\***"
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
df = []

dateLs = np.sort(dateLs)
dataPathLs = np.sort(dataPathLs)
for thisDate, thisPath in zip(dateLs, dataPathLs):
    data = pd.read_pickle(thisPath)
    data = data.iloc[:, 1:]
    data = data.sort_values(by=['order', 'innerSeq'])
    if 'mrstaat' in data.columns:
        data["mrstaat"] = data.groupby(['order'])['mrstaat'].transform('first')
        data["mrstauc"] = data.groupby(['order'])['mrstauc'].transform('first')
        data["mrsb90"] = data.groupby(['order'])['mrsb90'].transform('first')
        data["mrss90"] = data.groupby(['order'])['mrss90'].transform('first')
        data["aaa"] = data.groupby(['order'])['aaa'].transform('first')
    data["ars"] = data.groupby(['order'])['ars'].transform('first')
    data["sdd"] = data.groupby(['order'])['sdd'].transform('first')
    
    data = data[data['updateType'] == 4]
    print('1. Loading Data ' + thisDate )
    if data[data['absFilledThisUpdate'] == 0].shape[0] != 0:
        print('There are stocks with zero trade size')
        print(data[data['absFilledThisUpdate'] == 0].groupby(['colo', 'accCode'])['secid'].size())
        data = data[data['absFilledThisUpdate'] != 0]
    if data[data['beta_60'].isnull()].shape[0] != 0:
        print('There are stocks with null beta')
        print(data[data['beta_60'].isnull()])
        data = data[~data['beta_60'].isnull()]
    data['max_trade'] = data.groupby('order')['absOrderSizeCumFilled'].transform('max')
    data['last'] = 0
    data.loc[data['max_trade'] == data['absOrderSizeCumFilled'], 'last'] = 1

    data["buyRet"] = np.where(data["orderDirection"].isin([1, 2]), data["adjMid_F90s"] / data["tradePrice"] - 1, np.nan)
    data["buyRet1"] = np.where(data["orderDirection"].isin([1, 2]), data["adjMid_F300s"] / data["tradePrice"] - 1, np.nan)
    data["sellRet"] = np.where(data["orderDirection"].isin([-1, -2]), data["tradePrice"] / data["adjMid_F90s"] - 1, np.nan)
    data["sellRet1"] = np.where(data["orderDirection"].isin([-1, -2]), data["tradePrice"] / data["adjMid_F300s"] - 1, np.nan)
    data["indexRet"] = data["indexClose_F90s"]/data["indexClose"] - 1
    data["indexRet1"] = data["indexClose_F300s"]/data["indexClose"] - 1
    data["buyAlpha"] = np.where(data["orderDirection"].isin([1, 2]), data["buyRet"] - data["indexRet"] * data["beta_60"], np.nan)
    data["buyAlpha1"] = np.where(data["orderDirection"].isin([1, 2]), data["buyRet1"] - data["indexRet1"] * data["beta_60"], np.nan)
    data["sellAlpha"] = np.where(data["orderDirection"].isin([-1, -2]), data["sellRet"] + data["indexRet"] * data["beta_60"], np.nan)
    data["sellAlpha1"] = np.where(data["orderDirection"].isin([-1, -2]), data["sellRet1"] + data["indexRet1"] * data["beta_60"], np.nan)
    data["buyNum"] = np.where((data["orderDirection"].isin([1, 2])) & (data['last'] == 1), 1, 0)
    data["sellNum"] = np.where((data["orderDirection"].isin([-1, -2])) & (data['last'] == 1), 1, 0)
    data['server_account'] = data['colo'].apply(lambda x: x.replace('_', '')) + '_' + data['accCode'].astype(str)
    df += [data]
df = pd.concat(df, sort=False)

print('2. Orders with nan ars in updateType 0')
if df[df['ars'].isnull()].shape[0] != 0:
    print(df[df['ars'].isnull()])
    df = df[~df['ars'].isnull()]
df.loc[(df['date'] <= 20201013) & (df['ars'].isin([121, 131])), 'mrstauc'] = 1000
df.loc[(df['date'] <= 20201013) & (df['ars'].isin([221, 231])), 'mrstauc'] = 2000
df.loc[(df['date'] <= 20201013) & (df['ars'].isin([321, 331])), 'mrstauc'] = 3000
df.loc[(df['date'] <= 20201013) & (~df['ars'].isin([121, 221, 321, 131, 231, 331])), 'mrstauc'] = 0
df.loc[(df['date'] <= 20201013) & (df['mrstauc'].isin([1000, 2000, 3000])), 'mrstaat'] = 3000
df.loc[(df['date'] <= 20201013) & (df['mrstauc'] == 0), 'mrstaat'] = 1000

print('3. Current strategy status')
if 2 in df['mrstauc'].unique():
    print(df[df['mrstauc'] == 2].groupby('mrstaat')['mrstauc'].unique())
    df.loc[df['mrstauc'] == 2, 'mrstauc'] = 0
print(df.groupby('mrstaat')['mrstauc'].unique())

print('4. Strange strategy')
if df[(df["mrstaat"].isnull()) | (df['mrstaat'] == 0) | (df['mrstaat'] == 2000)].shape[0] != 0:
    print('1) null mrstaat value')
    print(df[(df["mrstaat"].isnull())][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'ars']])
    print(df[(df["mrstaat"].isnull())].groupby(['date', 'colo', 'accCode'])['secid'].size())
    print('2) 0 mrstaat value')
    print(df[(df["mrstaat"] == 0)][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'ars']])
    print('3) 2000 mrstaat value')
    print(df[(df["mrstaat"] == 2000)][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'ars']])
df = df[(~df["mrstaat"].isnull()) & (df['mrstaat'] != 0) & (df['mrstaat'] != 2000)]

if 'mrsb90' in df.columns:
    df['m1'] = df['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
    df['m2'] = df['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
    if df[df['mrsb90'] == '-'].shape[0] != 0:
        print('There are ticks with mrsb90 value -')
        print(df[df['mrsb90'] == '-'][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'aaa', 'ars']])
        df = df[df['mrsb90'] != '-']
    if df[df['aaa'] == '-'].shape[0] != 0:
        print('There are ticks with aaa value -')
        print(df[df['aaa'] == '-'][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'aaa', 'ars']])
        df = df[df['aaa'] != '-']   
    df['mrsb90'] = df['mrsb90'].astype(float)
    df['aaa'] = df['aaa'].astype(float)
    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'mrstauc'] = \
    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'm2']

    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'mrstaat'] = \
    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'm1']

    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'mrstauc'] = \
    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'm2']

    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'mrstaat'] = \
    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'm1']   
df.groupby(['mrstaat', 'mrstauc'])['date'].size()

1. Loading Data 20201204
1. Loading Data 20201207
There are stocks with null beta
        aaa  absFilledThisUpdate  absOrderSize  absOrderSizeCumFilled  \
5358    0.0                112.0         112.0                  112.0   
17459   0.0                112.0         112.0                  112.0   
33748   0.0                112.0         112.0                  112.0   
46400   0.0                112.0         112.0                  112.0   
59842   0.0                112.0         112.0                  112.0   
64334   0.0                112.0         112.0                  112.0   
88413   0.0                112.0         112.0                  112.0   
100565  0.0                112.0         112.0                  112.0   
113534  0.0                 93.0         112.0                   93.0   
113535  0.0                 19.0         112.0                  112.0   
132169  0.0                112.0         112.0                  112.0   
138994  0.0                112.0         1

2. Orders with nan ars in updateType 0
3. Current strategy status
mrstaat
1000.0                                                 [0.0]
3000.0                              [1000.0, 2000.0, 3000.0]
11000.0                          [10000.0, 30000.0, 20000.0]
13000.0    [11000.0, 21000.0, 12000.0, 23000.0, 32000.0, ...
Name: mrstauc, dtype: object
4. Strange strategy


C:\Users\win\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


mrstaat  mrstauc
1000.0   0.0         59451
3000.0   1000.0     285133
         2000.0      22990
         3000.0       9693
11000.0  10000.0      1050
         20000.0       123
         30000.0        61
13000.0  11000.0     25132
         12000.0      1294
         13000.0       435
         21000.0       801
         22000.0       916
         23000.0       183
         31000.0       438
         32000.0       367
         33000.0       477
Name: date, dtype: int64

In [119]:
test = pd.concat([orderLog[(orderLog['accCode'] == 5456) & (orderLog['ars'].isin([1, 7]))], orderLog[(orderLog['accCode'].isin([9758, 9441, 9461])) & (orderLog['ars'].isin([1, 7])) & (orderLog['date'] == 20201207)]]).reset_index(drop=True)
re1 = test[test['updateType'] == 0].groupby(['date', 'accCode'])['order'].size().reset_index()
re1 = re1.rename(columns={'order':'totalNum'})
re2 = test[(test['updateType'] == 0) & (test['ars'] == 7)].groupby(['date', 'accCode'])['order'].size().reset_index()
re2 = re2.rename(columns={'order':'IFRNum'})
re = pd.merge(re2, re1, on=['date', 'accCode'])
re['perc(%)'] = re['IFRNum'] / re['totalNum']
re['perc(%)'] = re['perc(%)'].apply(lambda x: '%.f'%(x * 100))
HTML(re.groupby(['date', 'accCode']).first().to_html())

In [120]:
test['tag'] = np.where(test['ars'] == 1, 'normal taking orders', np.where(
test['ars'] == 7, 'IFR taking orders', 'else'))
display(test[test['tag'] == 'else'])
re1 = test[test['updateType'] == 0].groupby(['date', 'tag', 'accCode'])['order'].size().reset_index()
re1 = re1.rename(columns={'order':'numOfTotalOrders'})
re2 = test[test['updateType'] == 1].groupby(['date', 'tag', 'accCode'])['order'].size().reset_index()
re2 = re2.rename(columns={'order':'numOfCancelOrders'})
re = pd.merge(re1, re2, on=['date', 'tag', 'accCode'], how='left')
re = re.fillna(0)
re['fillRate(%)'] = 1 - re['numOfCancelOrders'] / re['numOfTotalOrders']
re['fillRate(%)'] = re['fillRate(%)'].apply(lambda x: '%.1f'%(x * 100))
HTML(re.groupby(['date', 'accCode', 'tag']).first().to_html())

,ApplSeqNum,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,test,broker,colo_broker,order,group,startClock,duration,orderDirection1,status,orderNtl,m1,m2,exchange,tradeNtl,sta,tag


In [52]:
df1 = df[df['accCode'] == 5456]
df1['tag'] = np.where(df1['ars'] == 1, 'normal taking orders', np.where(
df1['ars'] == 7, 'IFR taking orders', 'else'))
display(df1[df1['tag'] == 'else'])
df1['Side'] = np.where(df1['orderDirection'].isin([1, 2]), 'Buy', 'Sell')
df1['Ret'] = np.where(df1['Side'] == 'Buy', df1['buyRet'] * 10000, df1['sellRet'] * 10000)
df1['Ret1'] = np.where(df1['Side'] == 'Buy', df1['buyRet1'] * 10000, df1['sellRet1'] * 10000)
re1 = df1.groupby(['Side', 'tag'])['Ret'].describe()[['mean', 'std', 'min', '25%', '50%', '75%', 'max']].reset_index()
ree1 = df1.groupby(['Side', 'tag'])['order'].nunique().reset_index()
ree1 = ree1.rename(columns={'order':'num'})
re1 = pd.merge(re1, ree1, on=['Side', 'tag'])
re1['Ret'] = '1. F90s_ret'
re2 = df1.groupby(['Side', 'tag'])['Ret1'].describe()[['mean', 'std', 'min', '25%', '50%', '75%', 'max']].reset_index()
ree2 = df1.groupby(['Side', 'tag'])['order'].nunique().reset_index()
ree2 = ree2.rename(columns={'order':'num'})
re2 = pd.merge(re2, ree2, on=['Side', 'tag'])
re2['Ret'] = '2. F300s_ret'
re = pd.concat([re1, re2])
re = re.sort_values(by=['Side', 'Ret', 'num', 'tag'])
for i in ['mean', 'std', 'min', '25%', '50%', '75%', 'max']:
    re[i] = re[i].apply(lambda x: '%.2f'%(x))
from IPython.display import display, HTML
re.index = pd.MultiIndex.from_arrays([list(re['Side'].values), list(re['Ret'].values), list(re['num'].values), list(re['tag'].values)],
                                            names=['Side', 'Ret', 'num','tag'])
re.index = pd.MultiIndex.from_tuples([(x[0], x[1][3:], x[2], x[3]) for x in re.index],
                                            names=['Side', 'Ret', 'num','tag'])
HTML(re[['mean', 'std', 'min', '25%', '50%', '75%', 'max']].to_html())

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,test,order,firstUpdateType,insertNum,innerSeq,firstClock,indexCat,beta_60,mdStartClock,mdClockAtArrival,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose,indexClose_F30s,indexClose_F90s,indexClose_F300s,max_trade,last,buyRet,buyRet1,sellRet,sellRet1,indexRet,indexRet1,buyAlpha,buyAlpha1,sellAlpha,sellAlpha1,buyNum,sellNum,server_account,m1,m2,tag


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [125]:
df1 = pd.concat([df[(df['accCode'] == 5456) & (df['ars'].isin([1, 7]))], df[(df['accCode'].isin([9758, 9441, 9461])) & (df['ars'].isin([1, 7])) & (df['date'] == 20201207)]]).reset_index(drop=True)
df1['tag'] = np.where(df1['ars'] == 1, 'normal taking orders', np.where(
df1['ars'] == 7, 'IFR taking orders', 'else'))
display(df1[df1['tag'] == 'else'])
df1['tradeNtl'] = df1['tradePrice'] * df1['absFilledThisUpdate']
df1['Side'] = np.where(df1['orderDirection'].isin([1, 2]), 'Buy', 'Sell')
df1['stradeNtl'] = df1.groupby(['date', 'Side', 'accCode', 'tag'])['tradeNtl'].transform('sum')
df1['Ret'] = np.where(df1['Side'] == 'Buy', df1['buyRet'] * 10000, df1['sellRet'] * 10000)
df1['Ret1'] = np.where(df1['Side'] == 'Buy', df1['buyRet1'] * 10000, df1['sellRet1'] * 10000)
df1['90s_ret'] = df1['Ret'] * df1['tradeNtl']
df1['90s_ret']= df1.groupby(['date', 'Side', 'accCode', 'tag'])['90s_ret'].transform('sum')
df1['90s_ret']= df1['90s_ret'] / df1['stradeNtl']
df1['300s_ret'] = df1['Ret1'] * df1['tradeNtl']
df1['300s_ret']= df1.groupby(['date', 'Side', 'accCode', 'tag'])['300s_ret'].transform('sum')
df1['300s_ret']= df1['300s_ret'] / df1['stradeNtl']
re1 = df1.groupby(['date', 'Side', 'accCode', 'tag'])['order'].nunique().reset_index()
re1 = re1.rename(columns={'order':'numOfOrders'})
re2 = df1.groupby(['date', 'Side', 'accCode', 'tag'])['tradeNtl'].sum().reset_index()
re2 = re2.rename(columns={'tradeNtl':'notional'})
re2['notional_sum'] = re2.groupby(['date', 'Side', 'accCode'])['notional'].transform('sum')
re2['notionalPerc'] = (re2['notional'] / re2['notional_sum']) * 100
re3 = df1.groupby(['date', 'Side', 'accCode', 'tag'])['90s_ret'].first().reset_index()
re4 = df1.groupby(['date', 'Side', 'accCode', 'tag'])['300s_ret'].first().reset_index()
re = pd.merge(re1, re2, on=['date', 'Side', 'accCode', 'tag'])
re = pd.merge(re, re3, on=['date', 'Side', 'accCode', 'tag'])
re = pd.merge(re, re4, on=['date', 'Side', 'accCode', 'tag'])
re['notional'] = re['notional'].astype(int)
for cols in ['90s_ret', '300s_ret']:
    re[cols] = re[cols].apply(lambda x: '%.2f'%(x))
for cols in ['notionalPerc']:
    re[cols] = re[cols].apply(lambda x: '%.1f'%(x))
re = re[((re['accCode'] == 9441) & (re['Side'] == 'Sell')) | (re['accCode'] != 9441)]
HTML(re.groupby(['date', 'Side', 'accCode', 'tag'])[['numOfOrders', 'notional', 'notionalPerc', '90s_ret', '300s_ret']].first().to_html())

,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,test,order,firstUpdateType,insertNum,innerSeq,firstClock,indexCat,beta_60,mdStartClock,mdClockAtArrival,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose,indexClose_F30s,indexClose_F90s,indexClose_F300s,max_trade,last,buyRet,buyRet1,sellRet,sellRet1,indexRet,indexRet1,buyAlpha,buyAlpha1,sellAlpha,sellAlpha1,buyNum,sellNum,server_account,m1,m2,tag


In [126]:
725622 / (725622 + 10467368)

0.06482825411261871